# **`Building ChatBot with Message History`**

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() # this will import all the api keys in the production environment
groq_api_key = os.getenv('GROQ_API_KEY')

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It" , groq_api_key = groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002159F616FE0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002159F614B20>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage 
model.invoke([HumanMessage(content="hi, My name is Shahbaz")])

AIMessage(content="Hello Shahbaz, it's nice to meet you! 👋\n\nWhat can I do for you today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 16, 'total_tokens': 41, 'completion_time': 0.045454545, 'prompt_time': 0.002225719, 'queue_time': 0.292437952, 'total_time': 0.047680264}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--bbe7486d-68f8-4566-92aa-bc39f1a932e6-0', usage_metadata={'input_tokens': 16, 'output_tokens': 25, 'total_tokens': 41})

## Use of PromptTemplate (which is dynamic part of single message prompt type)``

In [4]:
from langchain_core.prompts import PromptTemplate
template = PromptTemplate(
    template="Greet this person in 5 languages. The name of the person is {name}",
    input_variables=['name']
)
prompt = template.invoke({'name' : 'Shahbaz'})
model.invoke(prompt)

AIMessage(content="Here are greetings for Shahbaz in 5 languages:\n\n1. **English:** Hello, Shahbaz!\n2. **Spanish:** Hola, Shahbaz!\n3. **French:** Bonjour, Shahbaz !\n4. **German:** Hallo, Shahbaz!\n5. **Japanese:** こんにちは、シャーバズさん (Konnichiwa, Shābaz-san) \n\n\nLet me know if you'd like greetings in any other languages! \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 25, 'total_tokens': 125, 'completion_time': 0.181818182, 'prompt_time': 0.002110179, 'queue_time': 0.261300922, 'total_time': 0.183928361}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--ce7065f8-c08d-41f5-aa69-5ed614ecb3a0-0', usage_metadata={'input_tokens': 25, 'output_tokens': 100, 'total_tokens': 125})

In [5]:
### Using History Of Messages
from langchain_core.messages import HumanMessage , AIMessage
model.invoke(
    [HumanMessage(content="Hi I am Mirza Shahbaz Baig and i am a full Stack Web + AI engineer")]
)

AIMessage(content="Hello Mirza Shahbaz Baig, it's nice to meet you!\n\nThat's a fantastic combination of skills to have - full stack web development and AI engineering.  \n\nWhat kind of projects are you working on these days? I'd love to hear more about your work!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 27, 'total_tokens': 89, 'completion_time': 0.112727273, 'prompt_time': 0.002159589, 'queue_time': 0.26103871, 'total_time': 0.114886862}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--2c8aba3b-afbd-41c2-9e96-cf3e18279251-0', usage_metadata={'input_tokens': 27, 'output_tokens': 62, 'total_tokens': 89})

In [6]:
model.invoke(
    [
        HumanMessage(content="Hi I am Mirza Shahbaz Baig and i am a full Stack Web + AI engineer"),
        AIMessage(content="That's great, Mirza Shahbaz Baig! \n\nIt's impressive to have expertise in both full-stack web development and AI engineering. Those are two rapidly growing fields that are increasingly intertwined. \n\nWhat kind of projects are you working on? Are you interested in a particular niche within those areas? I'd love to hear more about your work.\n"),
        HumanMessage(content="Do you Know my name and what do i do?")
    ]
)

AIMessage(content='As a large language model, I don\'t have memory of past conversations or personal information about individuals.  \n\nSo while you told me your name is Mirza Shahbaz Baig and that you are a full-stack web + AI engineer, I don\'t "know" that information in the way a person would. Every interaction we have is a fresh start.  \n\nIf you\'d like to tell me more about yourself or your work, I\'m happy to listen!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 101, 'prompt_tokens': 124, 'total_tokens': 225, 'completion_time': 0.183817184, 'prompt_time': 0.009016388, 'queue_time': 0.519029973, 'total_time': 0.192833572}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--b25b5d21-4163-4706-ba5d-b971d3d4a0d4-0', usage_metadata={'input_tokens': 124, 'output_tokens': 101, 'total_tokens': 225})

In [ ]:
### Message History
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory 
# class which provide the blueprint that includes the methods and attributes that all chathistories must include it just provide a standard for chatmsghistory which is the actual implementation of history storage
from langchain_core.runnables.history import RunnableWithMessageHistory
store = {}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory() # initialize the message history w.r.t to particular user of particular session id
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model , get_session_history) # Runnable is used for chaining the different components of langchain together

In [8]:
config = {
    "configurable": {
        "session_id": "chat1"
    }
}


In [9]:
response = with_message_history.invoke(
       [ HumanMessage(content = "Hi I am Mirza Shahbaz Baig and i am a full Stack Web + AI engineer")],
        config = config # we are interacting wrt to chat1 session id 
)

In [10]:
with_message_history.invoke(
    [HumanMessage(content="What is my name ?")],
    config = config
)

AIMessage(content='Your name is Mirza Shahbaz Baig.  \n\nI remembered it from our first interaction. 😊  Is there anything else I can help you with?  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 112, 'total_tokens': 147, 'completion_time': 0.063636364, 'prompt_time': 0.005252799, 'queue_time': 0.26092603600000003, 'total_time': 0.068889163}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--a2d15508-b647-4e56-878d-c220783d22dc-0', usage_metadata={'input_tokens': 112, 'output_tokens': 35, 'total_tokens': 147})

In [11]:
## Change the config give it different session id
config1 = {
    "configurable":{
        "session_id" : "chat12"
    }
}
with_message_history.invoke(
    [HumanMessage(content="What is my name ?")],
    config = config1
)

AIMessage(content="As a large language model, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 14, 'total_tokens': 57, 'completion_time': 0.078181818, 'prompt_time': 0.001907009, 'queue_time': 0.27247456999999997, 'total_time': 0.080088827}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--3a67f240-438d-407e-b95a-d5f8c0d3ead6-0', usage_metadata={'input_tokens': 14, 'output_tokens': 43, 'total_tokens': 57})

# **~Working With Prompt Template~**

In [12]:
from langchain_core.prompts import ChatPromptTemplate , MessagesPlaceholder
prompt = ChatPromptTemplate([
        ('system' , "You are an helpfull assistant answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name = "messages")
])
chain = prompt | model

In [13]:
chain.invoke({"messages" : [HumanMessage(content="hi my name is Mirza shahbaz Baig")]})

AIMessage(content="Hello Mirza Shahbaz Baig! \n\nIt's nice to meet you. I'm ready to answer your questions to the best of my ability.  \n\nWhat can I help you with today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 34, 'total_tokens': 82, 'completion_time': 0.087272727, 'prompt_time': 0.002304859, 'queue_time': 0.262209422, 'total_time': 0.089577586}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--c9291700-d441-4765-9053-58db1f503d13-0', usage_metadata={'input_tokens': 34, 'output_tokens': 48, 'total_tokens': 82})

In [14]:
with_message_history = RunnableWithMessageHistory(chain , get_session_history)

In [15]:
config = {
    "configurable":{
        "session_id" : "chat3"
    }
}

In [16]:
with_message_history.invoke(
    [HumanMessage(content="do you know about Jquery")],
    config= config
)

AIMessage(content="Yes, I do! \n\njQuery is a fast, small, and feature-rich JavaScript library. It simplifies many common tasks that developers face when working with JavaScript, making it easier to build interactive and dynamic web pages. \n\n**Here are some key things to know about jQuery:**\n\n* **DOM Manipulation:** jQuery makes it incredibly easy to select, modify, and create HTML elements. This is essential for building interactive interfaces, animations, and dynamic content updates.\n* **Event Handling:** jQuery provides a streamlined way to handle user interactions like clicks, hovers, and form submissions.\n* **Animations:**  jQuery offers a simple syntax for creating smooth and customizable animations, adding visual flair to your websites.\n* **AJAX:** jQuery simplifies making asynchronous requests to servers, allowing you to fetch data and update content without needing to reload the entire page.\n* **Cross-browser Compatibility:** jQuery works consistently across different 

In [17]:
## Adding More complexity 
from langchain_core.prompts import ChatPromptTemplate , MessagesPlaceholder
prompt = ChatPromptTemplate([
        ('system' , "You are an helpfull assistant answer all the question to the best of your ability into {language} language"),
        MessagesPlaceholder(variable_name = "messages")
])
chain = prompt | model

In [20]:
chain.invoke(
    {"messages" : [HumanMessage(content="Hi My name is Shahbaz")] , "language" : "Spanish"}
)

AIMessage(content='¡Hola Shahbaz!  \n\nEncantado de conocerte.  ¿Cómo puedo ayudarte hoy?  Estoy aquí para responder a todas tus preguntas en español lo mejor que pueda.  \n\n¡No dudes en preguntarme lo que necesites! 😄\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 34, 'total_tokens': 89, 'completion_time': 0.1, 'prompt_time': 0.003352397, 'queue_time': 0.258214444, 'total_time': 0.103352397}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--784da676-658d-4096-8aac-cc672856bea4-0', usage_metadata={'input_tokens': 34, 'output_tokens': 55, 'total_tokens': 89})

## Manage Conversation History 

In [23]:
from langchain_core.messages import SystemMessage , trim_messages
# trim message --> this is reduce function that apply on the message history with the chatbot and reduce the chat messages according to the limit of tokens that is given that helps to manage the context window of an LLM
trimmer = trim_messages(
    max_tokens = 70,
    strategy = "last",
    token_counter = model,
    include_system = True ,# never trim the system message
    allow_partial = False,
    start_on = "human"

)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [24]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain = (
    RunnablePassthrough.assign(messages = itemgetter("messages")|trimmer)
    | prompt 
    | model
)

In [29]:
response = chain.invoke({
    "messages" : messages + [HumanMessage(content="What math problem do i ask with you ?")] , 
    "language" : "English"
})


In [30]:
response.content

"I can do all sorts of math problems! \n\nHere are some ideas:\n\n* **Basic arithmetic:** Addition, subtraction, multiplication, division (even with decimals and fractions!)\n* **Algebra:** Solving for unknowns, simplifying expressions\n* **Calculus:** Derivatives and integrals (though I'm still learning those!)\n* **Logic problems:**  I can help you reason through word problems and puzzles.\n\nWhat are you in the mood for?\n"